In [296]:
import os
from os import walk
from bs4 import BeautifulSoup
import re
import pandas as pd

In [297]:
dir_path = os.path.join('.','AWR')
awr_paths = []
for (dir_path, dir_names, file_names) in walk(dir_path):
    file_paths = [os.path.join(dir_path, file_name) for file_name in file_names]
    awr_paths.extend(file_paths)

In [321]:
output = []
for awr_path in awr_paths:
    print(awr_path)
    with open(awr_path) as f:
        soup = BeautifulSoup(f)

    row = {}
    row['AWR file'] = awr_path

    table = soup.find(string=re.compile('DB Name')).findParent('table')
    df = pd.read_html(str(table))[0]
    row['DB Name'] = str(df['DB Name'][0])

    table = soup.find(string=re.compile('Instance')).findParent('table')
    df = pd.read_html(str(table))[0]
    row['Instance Name'] = str(df['Instance'][0])
 
    table = soup.find(string=re.compile('Host')).findParent('table')
    df = pd.read_html(str(table))[0]
    try:
        row['Host Name'] = str(df['Host Name'][0])
    except:
        row['Host Name'] = str(df['Host'][0])

    table = soup.find(string=re.compile('Elapsed:')).findParent('table')
    df = pd.read_html(str(table))[0]
    string = str(df[df.iloc[:,0] == 'Elapsed:']['Snap Time'])
    string = re.compile('\d*,?\d*\.\d*').search(string).group(0)
    string = string.replace(',','')
    row['Elapsed Time (mins)'] = float(string)

    string = str(df[df.iloc[:,0] == 'DB Time:']['Snap Time'])
    string = re.compile('\d*,?\d*\.\d*').search(string).group(0)
    string = string.replace(',','')
    row['DB Time (mins)'] = float(string)


    chapter = soup.find(string=re.compile('Wait Classes by Total Wait Time'))
    if chapter:
        table = chapter.find_next('table')
        df = pd.read_html(str(table))[0]
        element = df[df.iloc[:,0] == 'DB CPU']['Total Wait Time (sec)']
        element = element.replace({"K":"*1e3", "M":"*1e6", "G":"*1e9", ",":""}, regex=True).map(pd.eval).astype(float)
        row['DB CPU (s)'] = float(element)
    else:
        chapter = soup.find(re.compile("h\d+|p"), string=re.compile('Service Statistics'))
        table = chapter.find_next('table')
        df = pd.read_html(str(table))[0]
        column = df['DB CPU (s)']
        row['DB CPU (s)'] = float(column.sum())
    
    chapter = soup.find(string=re.compile('NUM_CPUS'))
    table = chapter.findParent('table')
    df = pd.read_html(str(table))[0]
    string = str(df[df.iloc[:,0] == 'NUM_CPUS'].iloc[0,1])
    string = string.replace(',','')
    row['CPUs'] = float(string)
    
    string = str(df[df.iloc[:,0] == 'NUM_CPU_CORES'].iloc[0,1])
    string = string.replace(',','')
    row['Cores'] = float(string)
    
    string = str(df[df.iloc[:,0] == 'PHYSICAL_MEMORY_BYTES'].iloc[0,1])
    string = string.replace(',','')
    row['Memory (GB)'] = float(string)/(1024**3)

    chapter = soup.find(string=re.compile('Instance CPU'))
    if chapter:
        table = chapter.find_next('table')
        df = pd.read_html(str(table))[0]
        row['%Busy CPU'] = float(df['%Busy CPU'])
    else:
        print('   error: "Instance CPU" not found')

    text = soup.find(string=re.compile('SGA use \(MB\):'))
    if text:
        table = text.findParent('table')
        df = pd.read_html(str(table))[0]
        row['SGA use (MB)'] = float(df[df.iloc[:,0] == 'SGA use (MB):']['End'])

        row['PGA use (MB)'] = float(df[df.iloc[:,0] == 'PGA use (MB):']['Begin'])
    else:
        print('   error: "SGA use (MB):" not found')

    chapter = soup.find(re.compile("h\d+|p"), string=re.compile('IOStat by Filetype summary'))
    if chapter:     
        table = chapter.find_next('table')
        df = pd.read_html(str(table))[0]
        element = df[df.iloc[:,0] == 'TOTAL:']['Data per sec']
        element = element.replace({"K":"*1e3", "M":"*1e6", "G":"*1e9", ",":""}, regex=True).map(pd.eval).astype(float)
        row['Read Throughput (MB/s)'] = float(element)/1000/1000
    
        element = df[df.iloc[:,0] == 'TOTAL:']['Data per sec.1']
        element = element.replace({"K":"*1e3", "M":"*1e6", "G":"*1e9", ",":""}, regex=True).map(pd.eval).astype(float)
        row['Write Throughput (MB/s)'] = float(element)/1000/1000

        element = df[df.iloc[:,0] == 'TOTAL:']['Reqs per sec']
        element = element.replace({"K":"*1e3", "M":"*1e6", "G":"*1e9", ",":""}, regex=True).map(pd.eval).astype(float)
        row['Read IOPS'] = float(element)

        element = df[df.iloc[:,0] == 'TOTAL:']['Reqs per sec.1']
        element = element.replace({"K":"*1e3", "M":"*1e6", "G":"*1e9", ",":""}, regex=True).map(pd.eval).astype(float)
        row['Write IOPS'] = float(element)
    else:
        print('   error: "IOStat by Filetype summary" not found')

    output.append(row)

    #break

output = pd.DataFrame(output)
output['Total Throughput (MB/s)'] = output['Read Throughput (MB/s)'] + output['Write Throughput (MB/s)']
output['Total IOPS'] = output['Read IOPS'] + output['Write IOPS']
output['%DB Time of Elapsed Time'] = output['DB Time (mins)'] / output['Elapsed Time (mins)']
output['CPU total capacity (s)'] = output['Elapsed Time (mins)'] * 60 * output['CPUs']
output['%DB CPU of server capacity'] = output['DB CPU (s)']/output['CPU total capacity (s)']
output['ORA use (GB)'] = (output['SGA use (MB)'] + output['PGA use (MB)'])/1024
output['source CPU HT factor'] = output['CPUs'] / output['Cores']

output.to_excel(os.path.join('.', 'output', 'awr_data.xlsx'))

    

.\AWR\awr_report_ADTGT_10-11.html
.\AWR\awr_report_ADTGT_22-23.html
.\AWR\awr_report_CSDG4_10-11.html
.\AWR\awr_report_CSDG4_22-23.html
.\AWR\awr_report_CSIPG102_10-11.html
.\AWR\awr_report_CSIPG102_22-23.html
.\AWR\awr_report_CWSDG402_10-11.html
.\AWR\awr_report_CWSDG402_22-23.html
.\AWR\awr_report_PGOL_10-11.html
.\AWR\awr_report_PGOL_22-23.html
.\AWR\awr_report_SSCG1_10-11.html
   error: "Instance CPU" not found
   error: "SGA use (MB):" not found
   error: "IOStat by Filetype summary" not found
.\AWR\awr_report_SSCG1_22-23.html
   error: "Instance CPU" not found
   error: "SGA use (MB):" not found
   error: "IOStat by Filetype summary" not found


In [313]:
soup.find(string=re.compile('SGA use (MB):'))

In [299]:
awr_path = awr_paths[9]
with open(awr_path) as f:
    soup = BeautifulSoup(f)

In [302]:
table = soup.find(string=re.compile('Wait Classes by Total Wait Time'))

In [279]:
df

,Filetype Name,Reads: Data,Reqs per sec,Data per sec,Writes: Data,Reqs per sec.1,Data per sec.1,Small Read,Large Read
0,Data File,242.5G,45.36,34.447M,1003M,9.78,.139M,419.79us,4.49ms
1,Temp File,32.3G,42.26,4.594M,32.7G,43.40,4.641M,756.21us,1.18ms
2,Archive Log,14G,4.03,1.988M,1G,0.14,.144M,1.42ms,20.89ms
3,Log File,3.7G,4.92,.531M,1.9G,68.52,.27M,138.82us,10.82ms
4,Control File,2.3G,16.71,.331M,319M,2.59,.044M,107.14us,2.79ms
5,TOTAL:,295G,113.27,41.891M,36.9G,124.43,5.238M,483.06us,5.09ms


In [218]:
df['Data per sec'].replace({"K":"*1e3", "M":"*1e6", "G":"*1e9", ",":""}, regex=True).map(pd.eval).astype(float)
df['Data per sec']

0    37970000.0
1      922000.0
2     2027000.0
3      418000.0
4      378000.0
5       25000.0
6      151000.0
7           0.0
8           0.0
9    41890000.0
Name: Data per sec, dtype: float64

In [196]:

df[df.iloc[:,0] == 'NUM_CPUS'].iloc[0,1]

56

In [239]:
table = soup.find(string=re.compile('Wait Classes by Total Wait Time')).find_next('table')
df = pd.read_html(str(table))[0]
element = df[df.iloc[:,0] == 'DB CPU']['Total Wait Time (sec)']
element = element.replace({"K":"*1e3", "M":"*1e6", "G":"*1e9", ",":""}, regex=True).map(pd.eval).astype(float)
float(element)

1343.0

In [49]:
df = pd.read_html(str(table))[0]
df

,Wait Class,Waits,Total Wait Time (sec),Avg Wait Time,% DB time,Avg Active Sessions
0,Application,8902.0,1440,161.72ms,21.1,0.2
1,DB CPU,NaN,1343,NaN,19.7,0.2
2,System I/O,638477.0,728,1.14ms,10.7,0.1
3,User I/O,368901.0,483,1.31ms,7.1,0.1
4,Other,1678386.0,419,249.71us,6.1,0.1
5,Commit,173034.0,416,2.41ms,6.1,0.1
6,Cluster,239721.0,191,798.80us,2.8,0.0
7,Administrative,114856.0,133,1.16ms,1.9,0.0
8,Network,3828808.0,29,7.70us,0.4,0.0
9,Concurrency,325668.0,26,78.72us,0.4,0.0


In [59]:
float(df[df['Wait Class'] == 'Application']['Waits'] )

8902.0